In [10]:
from imp import reload
from nltk.corpus import stopwords
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib as mpl
import nltk,re,pprint
import sys,glob,os
import operator, string, argparse, math

In [111]:
swords = set(stopwords.words('english'))

def getallfilenames(foldername):#returns the name of all files inside the source folder. 		
    owd = os.getcwd()
    fld = foldername + "/"
    os.chdir(fld)					#this is the name of the folder from which the file names are returned.
    fnamearr = []						#empty array, the names of files are appended to this array, and returned.
    for file in glob.glob("*.txt"):
        fnamearr.append(file)
    os.chdir(owd)
    return fnamearr

def drawProgressBar(percent, barLen = 50):			#just a progress bar so that you dont lose patience
    sys.stdout.write("\r")
    progress = ""
    for i in range(barLen):
        if i<int(barLen * percent):
            progress += "="
        else:
            progress += " "
    sys.stdout.write("[ %s ] %.2f%%" % (progress, percent * 100))
    sys.stdout.flush()
    
def rem_stop_punct(originalText):
    splittedText = originalText.split()
    lenl = len(splittedText)
    wordFiltered = []
    tSent = []
    for r in range(lenl):
        wordx_1 = splittedText[r]
        wordx_2 = "".join(c for c in wordx_1 if c not in ('!','.',':',',','?',';','``','&','-','"','(',')','[',']','0','1','2','3','4','5','6','7','8','9')) 
        sWord = wordx_2.lower()
        if sWord not in swords:
            tSent.append(sWord)
    return " ".join(tSent)
            
#allfnames = getallfilenames("/Users/arnabborah/Documents/repositories/textclusteringDBSCAN/processFiles/trainCatFiles")
#allfnames

In [5]:
class dataProcessor:
    def __init__(self, fname, keep_factors = ['Job Description', 'Company Name', 'Industry'], group_column = 'Industry'):
        self.dataInitial = pd.read_csv(fname, encoding="latin")
        self.dataInitialSmall = self.dataInitial[['Job Description', 'Company Name', 'Industry']]
        self.swords = set(stopwords.words('english'))
        #print(len(self.swords),"stopwords present!")
        self.dataInitialGrouped = self.dataInitialSmall.groupby([group_column]).count()
        pd.set_option('display.max_rows', 50)
        print(self.dataInitialGrouped.sort_values(by=['Job Description'], ascending=False))

    # pipeline for purifying the text, write-pipeline, so just output filename can be provided
    def rem_stop_punct(self,originalText, ofilename):
        splittedText = originalText.split()
        lenl = len(splittedText)
        print("Length is: ",lenl, splittedText[:5])
        ofile = open(ofilename,'a')
        
        for r in range(lenl):
            linex = splittedText[r]
            linex2 = "".join(c for c in linex if c not in ('!','.',':',',','?',';','``','&','-','"','(',')','[',']','0','1','2','3','4','5','6','7','8','9'))
            linex3 = linex2.split()
            #prog=(r+1)/len(rawlines)
            for s in range(len(linex3)):
                noword = linex3[s].lower()
                if noword not in self.swords:
                    ofile.write(noword)
                    ofile.write(" ")
                        
os.chdir("/Users/arnabborah/Documents/repositories/textclusteringDBSCAN/scripts/")

# using both the classes declared above on a new dataset
rp = dataProcessor("../datasets/DataAnalyst.csv")

                                  Job Description  Company Name
Industry                                                       
-1                                            353           352
IT Services                                   325           325
Staffing & Outsourcing                        323           323
Health Care Services & Hospitals              151           151
Consulting                                    111           111
...                                           ...           ...
Chemical Manufacturing                          1             1
Pet & Pet Supplies Stores                       1             1
Consumer Product Rental                         1             1
Metals Brokers                                  1             1
News Outlet                                     1             1

[89 rows x 2 columns]


In [ ]:
class flingTFIDF:
    def __init__(self,data,cname):
        self.idfMatrix = {}
        self.distanceMatrix = {}
        self.termsforIDF = []
        self.cname = cname
        self.data = data
        self.lenv = len(self.data)
        
    def smartTokenizeColumn(self):
        self.stopsRemoved = []
        for index, row in self.data.iterrows():
            prog=(index+1)/self.lenv
            originText = row[self.cname]
            sentx = rem_stop_punct(originText)
            #self.stopsRemoved.append(sentx)
            drawProgressBar(prog)
            self.data.loc[index,'stopsRemoved'] = sentx
        #self.data['stopsRemoved'] = self.stopsRemoved
        self.cname = 'stopsRemoved'
        
    def getTF(self):
        print("adding term frequency column based on",self.cname)
        #self.tfList = []
        #self.tfShapes = []
        for index, row in self.data.iterrows():
            words_in_column = row[self.cname].split()
            if len(words_in_column)!=0:
                counts_all = Counter(words_in_column)
                words, count_values = zip(*counts_all.items())
                values_sorted, words_sorted = zip(*sorted(zip(count_values, words), key=operator.itemgetter(0), reverse=True))
                countdf = pd.DataFrame({'word': words_sorted, 'tf': values_sorted})
                #self.tfList.append(countdf)
                #self.tfShapes.append(countdf.shape)
                self.data.loc[index,'tfMatrix'] = countdf
            else:
                #self.tfList.append(pd.DataFrame(columns = ['word','tf']))
                #self.tfShapes.append("empty")
                self.data.loc[index,'tfMatrix'] = pd.DataFrame(columns = ['word','tf'])
            prog=(index+1)/self.lenv
            drawProgressBar(prog)
        #self.data['tfShape'] = self.tfShapes
        #self.data['tfList'] = self.tfList
        
    def getTFIDF(self):
        print("\nComputing and adding TF-IDF column based on",self.cname)
        for index, row in self.data.iterrows():
            tfmatrixThisrow = row['tfMatrix']
            tempTFIDF = []
            for indx, rwx in tfmatrixThisrow.iterrows():
                trmx = rwx['word']
                tfx = rwx['tf']
                idfx = self.idfMatrix[trmx]
                tfidfx = tfx*idfx
                #tempTFIDF.append(tfidfx)
                tfmatrixThisrow.loc[index,'tf-idf'] = tfidfx
            #tfmatrixThisrow['tf-idf'] = tempTFIDF
            #sumtfidf = tfmatrixThisrow['tf-idf'].sum() 
            prog=(index+1)/self.lenv
            drawProgressBar(prog)
                
    def computeIDFlistofterms(self):
        totalwords = 0
        print("\nComputing list of words for IDF...\n")
        for index, row in self.data.iterrows():
            words_in_column = set(row[self.cname].split())  
            for word in words_in_column:
                if word not in self.termsforIDF:
                    self.termsforIDF.append(word)
                    totalwords+=1
        print("Created list of terms for IDF matrix with", totalwords," terms.")     
        
    def getIdf(self,term):
        countPresentDocs = 0
        lenidf = len(self.termsforIDF)
        for i in range(lenidf):
            tfx = self.getTermFreq(i,term)
            if tfx>0:
                countPresentDocs+=1
            prog=(i+1)/lenidf
            drawProgressBar(prog)
        return countPresentDocs
        
    def computeIDFmatrix(self):
        self.computeIDFlistofterms()
        print("\nComputing global IDF matrix...\n")
        for term in self.termsforIDF:
            self.idfMatrix[term]=0
        for index, row in self.data.iterrows():
            listofterms = list(self.data['tfMatrix'][index]['word'])
            for term in listofterms:
                self.idfMatrix[term]=self.idfMatrix[term]+1
            prog=(index+1)/self.lenv
            drawProgressBar(prog)
        for term in self.termsforIDF:
            idfx = self.idfMatrix[term]          
            idfy = self.lenv/float(1+idfx)
            idfz = math.log(idfy,10)
            self.idfMatrix[term] = idfz
            
    def showData(self):
        print(self.data['tfMatrix'])
        
    def createDistanceMetadata(self):
        #sumList = []
        for index, row in self.data.iterrows():
            tfmatrixThisrow = row['tfMatrix']
            sumTFIDF = tfmatrixThisrow['tf-idf'].sum()
            #sumList.append({'sumTFIDF':sumTFIDF})
            self.data.loc[index,'sumTFIDF'] = sumTFIDF
              
    def distanceBtnTwoDocs(self, docId_1, docId_2):
        listWords_1 = set(list(self.data['tfMatrix'][docId_1]['word']))
        listWords_2 = set(list(self.data['tfMatrix'][docId_2]['word']))
        common = listWords_1.intersection(listWords_2)
        diff1_2 = listWords_1.difference(listWords_2)
        diff2_1 = listWords_2.difference(listWords_1)
        sumwt1 = self.data['sumTFIDF'][docId_1]
        sumwt2 = self.data['sumTFIDF'][docId_2]
        score_common, score_doc1, score_doc2 = 0,0,0
        for word_c in common:
            score_1 = float(self.data['tfMatrix'][docId_1].loc[self.data['tfMatrix'][docId_1]['word'] == word_c]['tf-idf'])
            score_2 = float(self.data['tfMatrix'][docId_2].loc[self.data['tfMatrix'][docId_2]['word'] == word_c]['tf-idf'])
            score_common += abs(score_1/float(sumwt1) - score_2/float(sumwt2))
        for word_d12 in diff1_2:
            score_1 = float(self.data['tfMatrix'][docId_1].loc[self.data['tfMatrix'][docId_1]['word'] == word_d12]['tf-idf'])
            score_doc1 += score_1/float(sumwt1)
        for word_d21 in diff2_1:
            score_2 = float(self.data['tfMatrix'][docId_2].loc[self.data['tfMatrix'][docId_2]['word'] == word_d21]['tf-idf'])
            score_doc2 += score_2/float(sumwt2)
        score_total = score_common + score_doc1 + score_doc2
        return(score_total)
    
    def computeDistanceBtnAllDocs(self):
        for j in range(100):
            for k in range(10):
                numx = j*10+k
                dist = self.distanceBtnTwoDocs(j,k)
                self.distanceMatrix[(j,k)] = dist
                prog=(numx+1)/1000
                drawProgressBar(prog)
                
        print(self.distanceMatrix[:10])       

In [234]:
a = set(list(ftf.data['tfMatrix'][1]['word']))
b = set(list(ftf.data['tfMatrix'][2]['word']))
common = a.intersection(b)
diff = a.difference(b)
print(len(a),len(b),len(common),len(diff))

256 281 49 207


In [272]:
float(ftf.data['tfMatrix'][1].loc[ftf.data['tfMatrix'][1]['word'] == 'reporting']['tf-idf'].astype(float))

1.554869363760395

In [283]:
def getDiff(data, docId_1, docId_2):
    listWords_1 = set(list(data['tfMatrix'][docId_1]['word']))
    listWords_2 = set(list(data['tfMatrix'][docId_2]['word']))
    common = listWords_1.intersection(listWords_2)
    diff1_2 = listWords_1.difference(listWords_2)
    diff2_1 = listWords_2.difference(listWords_1)
    sumwt1 = data['sumTFIDF'][docId_1]
    sumwt2 = data['sumTFIDF'][docId_2]
    score_common, score_doc1, score_doc2 = 0,0,0
    for word_c in common:
        score_1 = float(data['tfMatrix'][docId_1].loc[data['tfMatrix'][docId_1]['word'] == word_c]['tf-idf'])
        score_2 = float(data['tfMatrix'][docId_2].loc[data['tfMatrix'][docId_2]['word'] == word_c]['tf-idf'])
        score_common += abs(score_1/float(sumwt1) - score_2/float(sumwt2))
    for word_d12 in diff1_2:
        score_1 = float(data['tfMatrix'][docId_1].loc[data['tfMatrix'][docId_1]['word'] == word_d12]['tf-idf'])
        score_doc1 += score_1/float(sumwt1)
    for word_d21 in diff2_1:
        score_2 = float(data['tfMatrix'][docId_2].loc[data['tfMatrix'][docId_2]['word'] == word_d21]['tf-idf'])
        score_doc2 += score_2/float(sumwt2)
    score_total = score_common + score_doc1 + score_doc2
    return(score_total)


1.847963208154384
1.908929885185095
1.8723360728153131
1.8859878543870687
0.0
1.7567946040259788
1.8183410691594477
1.8871896618138921
1.7567946040259788
0.0
1.8346014378548616
1.908480190818862
1.8183410691594477
1.8346014378548616
0.0
1.8784864100019139


In [ ]:
# primary distance module run
ftf = flingTFIDF(rp.dataInitialSmall,'Job Description')
ftf.smartTokenizeColumn()
#ftf.getTF()
#ftf.computeIDFmatrix()
#ftf.getTFIDF()
#ftf.createDistanceMetadata()
#ftf.computeDistanceBtnAllDocs()

In [219]:
pd.set_option('display.max_rows', 500) 
total = ftf.data['distanceMetadata'][:10]
print(total)

0     {'sumTFIDF': 811.5693284137141}
1     {'sumTFIDF': 415.2875832148055}
2    {'sumTFIDF': 439.81593224954963}
3      {'sumTFIDF': 569.217930688834}
4    {'sumTFIDF': 420.10671852390715}
5    {'sumTFIDF': 289.92465152113436}
6     {'sumTFIDF': 304.3672379088695}
7    {'sumTFIDF': 190.85385731392748}
8     {'sumTFIDF': 375.1300840846757}
9     {'sumTFIDF': 392.1123815118852}
Name: distanceMetadata, dtype: object


In [ ]:
ftf.distanceMatrix